Let's start cleaning data!

In [38]:
import numpy as np
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/BL-Flickr-Images-Book.csv')

#Let's explore the data set
data.head()

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8287 entries, 0 to 8286
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Identifier              8287 non-null   int64  
 1   Edition Statement       773 non-null    object 
 2   Place of Publication    8287 non-null   object 
 3   Date of Publication     8106 non-null   object 
 4   Publisher               4092 non-null   object 
 5   Title                   8287 non-null   object 
 6   Author                  6509 non-null   object 
 7   Contributors            8287 non-null   object 
 8   Corporate Author        0 non-null      float64
 9   Corporate Contributors  0 non-null      float64
 10  Former owner            1 non-null      object 
 11  Engraver                0 non-null      float64
 12  Issuance type           8287 non-null   object 
 13  Flickr URL              8287 non-null   object 
 14  Shelfmarks              8287 non-null   

In [40]:
# data.describe() #not really useful here
data.columns

Index(['Identifier', 'Edition Statement', 'Place of Publication',
       'Date of Publication', 'Publisher', 'Title', 'Author', 'Contributors',
       'Corporate Author', 'Corporate Contributors', 'Former owner',
       'Engraver', 'Issuance type', 'Flickr URL', 'Shelfmarks'],
      dtype='object')

In [41]:
#Let's remove the info that we don't need - as they take up space and bog down runtime.
#Moreover my df will be easier to read and investigate

to_drop_columns = ['Edition Statement','Contributors','Corporate Author', 'Corporate Contributors', 'Former owner',
       'Engraver', 'Issuance type', 'Shelfmarks']

data.drop(to_drop_columns, axis=1, inplace=True)
#inplace = True will apply the change to current df

# or we can call: data.drop(columns=to_drop, inplace=True)

#### NOTE: 
If you know in advance which columns you’d like to retain, another option is to pass them to the **usecols** argument of **pd.read_csv**.

Let's change the index of the df. we can check if the identifier is unique and use it as index

In [42]:
data['Identifier'].is_unique

True

In [43]:
data = data.set_index(data.Identifier) #instead of assigning we could have use inplace=Ture
data.drop('Identifier',axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8287 entries, 206 to 4160339
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Place of Publication  8287 non-null   object
 1   Date of Publication   8106 non-null   object
 2   Publisher             4092 non-null   object
 3   Title                 8287 non-null   object
 4   Author                6509 non-null   object
 5   Flickr URL            8287 non-null   object
dtypes: object(6)
memory usage: 453.2+ KB


In [44]:
data['Date of Publication'] #let's explore this info

Identifier
206        1879 [1878]
216               1868
218               1869
472               1851
480               1857
              ...     
4158088           1838
4158128       1831, 32
4159563      [1806]-22
4159587           1834
4160339        1834-43
Name: Date of Publication, Length: 8287, dtype: object

#### Reflaction on Date of Publication:
A particular book can have only one date of publication. Therefore, we need to do the following:

- Remove the extra dates in square brackets, wherever present: 1879 [1878]
- Convert date ranges to their “start date”, wherever present: 1860-63; 1839, 38-54
- Completely remove the dates we are not certain about and replace them with NumPy’s NaN: [1897?]
- Convert the string nan to NumPy’s NaN value
- Synthesizing these patterns, we can actually take advantage of a single regular expression to extract the publication year:
**regex = r'^(\d{4})'**

In [45]:
#checking for uncretain date i.e. with ? symbol

data[data['Date of Publication'].str.contains('\?')==True] #if I don't use loc I can only filter by row or columns separatly
data.loc[data['Date of Publication'].str.contains('\?')==True,'Date of Publication'] 

#using loc I can just focus on a single column


Identifier
5385       [1897?]
5389       [1897?]
14466      [1860?]
51125      [1820?]
75692      [1762?]
            ...   
3857689     1830?]
3861858    [1868?]
3898588    [1858?]
3934491    [1899?]
3998913    [1838?]
Name: Date of Publication, Length: 119, dtype: object

In [46]:
uncertain_dates = data.loc[data['Date of Publication'].str.contains('\?')==True,'Date of Publication'] #checking nbr of uncretain dates
uncertain_dates.count()

119

In [47]:
print('number of rows:')
print(len(data))
print('number of uncertain dates:')
print(data.loc[data['Date of Publication'].str.contains('\?')==True,'Date of Publication'].count())
print('number of NaN')
print(data['Date of Publication'].isnull().sum())
data.loc[data['Date of Publication'].str.contains(r'^\[')==True,'Date of Publication']

number of rows:
8287
number of uncertain dates:
119
number of NaN
181


Identifier
5385         [1897?]
5389         [1897?]
11361       [1894-96
13364         [1885]
14466        [1860?]
             ...    
4003256       [1850]
4006300       [1866]
4112839      [1845.]
4114889       [1868]
4159563    [1806]-22
Name: Date of Publication, Length: 786, dtype: object

In [48]:
extr = data['Date of Publication'].str.extract(r'^(\d{4})', expand=False) #uncertain dates have been converted in nan
extr[uncertain_dates].unique()
extr

Identifier
206        1879
216        1868
218        1869
472        1851
480        1857
           ... 
4158088    1838
4158128    1831
4159563     NaN
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8287, dtype: object

In [49]:
extr.isnull().sum()

971

In [50]:
#let's check the values excluded from this extraction:
data.loc[data['Date of Publication'].str.contains(r'^\d{4}')==False,'Date of Publication'].unique()

array(['[1897?]', '[1894-96', '[1885]', '[1860?]', '[1833]', '[1817.]',
       '[1834]', '[1860,] 1861-1863', '[1872]', '[1874.]', '[1896]',
       '[1820?]', '[1894]', '[1879]', '[1898]', '[1762?]', '[1890]',
       '[1885?]', '[1785.]', '[1880?]', '[1885.]', '[1893]', '[1855.]',
       '[1872]]', '[1858.]', '[1836?]', '[1877]]', '[1869.]', '[1888]',
       '[1860.]', '[1879.]', '[1880.]', '[1852]', '[1866.]', '[1886]',
       '[1891]', '[1892]', '[1889]', '[1880]', '[1850?]', '[1846]',
       '[1800.]', '[1710?]', '[1782.]', '[1868-70.]', '[1851]', '[1836]',
       '[1875?]', '[1890.]', '[1807]', '[1842]', '[1897]', '[1892-1900]',
       '[1883.]', '[1866-68.]', '[1810?]', '[1824.]', '[1801.]', '[1849]',
       '[1846, 47.]', '[1835?]', '[1866-1867]', '[1886?]', '[1883]',
       '[1892.]', '[1844.]', '[1873-76.]', '[1899.]', '[1837-39]',
       '[1865.]', '[1889.]', '[1897.]', '[1886.]', '[1851.]', '[1878]',
       '[1844-47]', '[c. 1820.]', '[1848?]', '[1848.]', '[1818]',
       '[1

It seems that this extraction has removed all intervals - not sure this is the right thing! Let's include the first element of the interval [] but we still want to exclude the uncertain dates i.e. dates with ?

In [51]:
extr2 = data['Date of Publication'].str.extract(r'^\[?(\d{4})[^?]?', expand=False) #uncertain dates have been converted in nan
extr2

Identifier
206        1879
216        1868
218        1869
472        1851
480        1857
           ... 
4158088    1838
4158128    1831
4159563    1806
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8287, dtype: object

In [52]:
data['Date of Publication'] = pd.to_numeric(extr2, downcast='integer').fillna(0).astype(np.int64)
data['Date of Publication']

Identifier
206        1879
216        1868
218        1869
472        1851
480        1857
           ... 
4158088    1838
4158128    1831
4159563    1806
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8287, dtype: int64

In [53]:
#checks missing data
data['Date of Publication'].isnull().sum()/len(data)
print(f"Number of missing records: {data['Date of Publication'].isnull().sum() } over {len(data)} \
as percentage: {data['Date of Publication'].isnull().sum()/len(data):.2%}")

Number of missing records: 0 over 8287 as percentage: 0.00%


In [54]:
data.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879,S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


In [55]:
data['Place of Publication'].unique()

array(['London', 'London; Virtue & Yorston',
       'pp. 40. G. Bryan & Co: Oxford, 1898', ...,
       'pp. viii. 64. J. Debrett: London, 1789', 'G. Eld: London, 1608',
       'Newcastle upon Tyne'], dtype=object)

In [56]:
place_publ = data['Place of Publication']
london_publ = place_publ.str.contains('London') 
oxford_publ = place_publ.str.contains('Oxford')
place_publ = np.where(london_publ,'London',np.where(oxford_publ,'Oxford',place_publ.str.replace('-',' ')))
place_publ

array(['London', 'London', 'London', ..., 'London', 'Newcastle upon Tyne',
       'London'], dtype=object)

In [57]:
import requests

url = 'https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/university_towns.txt'
response = requests.get(url)
city_txt = response.text
city_txt
filename = "university_towns_v1.txt"
file = open(filename, 'w')
file.write(city_txt)


26593

In [58]:
#we now want to clean the txt file and create a dataframe with state and city
university_city = []
with open('university_towns_v1.txt') as f:
    for line in f:
        #states are marked with [edit]
        if '[edit]' in line:
            state = line
        #if [edit] is missing then we have a city
        else:
            university_city.append((state,line))

print(university_city[:5])

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'), ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'), ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'), ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'), ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n')]


In [59]:
#let's create a dataframe from this array:
state_region = pd.DataFrame(university_city,columns=['State','Region Name'])
state_region

,State,Region Name
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n
...,...,...
512,Wisconsin[edit]\n,River Falls (University of Wisconsin–River Fal...
513,Wisconsin[edit]\n,Stevens Point (University of Wisconsin–Stevens...
514,Wisconsin[edit]\n,Waukesha (Carroll University)\n
515,Wisconsin[edit]\n,Whitewater (University of Wisconsin–Whitewater...


In [62]:
#we can create a function to remove the parenthesis and just keep the city/state name

def remove_parenthesis(item):
    if '[' in item:
        return item[:item.find('[')]
    elif '(' in item:
        return item[:item.find('(')]
    else:
        return item

# we can use pandas.applymap
# when we use map we can assign a df or array to map or a function. here is an example using a
# function to apply to each eleme nt

state_region = state_region.applymap(remove_parenthesis)
state_region

,State,Region Name
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [68]:
# Example loading a csv file and renaming columns
url2 = 'https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/olympics.csv'
olympics_data = pd.read_csv(url2, header=1) #we can specify the header line
print(olympics_data.columns)
olympics_data

Index(['Unnamed: 0', '? Summer', '01 !', '02 !', '03 !', 'Total', '? Winter',
       '01 !.1', '02 !.1', '03 !.1', 'Total.1', '? Games', '01 !.2', '02 !.2',
       '03 !.2', 'Combined total'],
      dtype='object')


,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Independent Olympic Participants (IOP) [IOP],1,0,1,2,3,0,0,0,0,0,1,0,1,2,3
143,Zambia (ZAM) [ZAM],12,0,1,1,2,0,0,0,0,0,12,0,1,1,2
144,Zimbabwe (ZIM) [ZIM],12,3,4,1,8,1,0,0,0,0,13,3,4,1,8
145,Mixed team (ZZX) [ZZX],3,8,5,4,17,0,0,0,0,0,3,8,5,4,17


In [80]:
# renaming columns - let's create a dictionary for the columns names:
col_names = {}
new_names = ['Country','Summer Olympics','Gold','Silver','Bronze','Winter Olympics','Gold.1','Silver.1','Bronze.1',
            '# Games','Gold.2','Silver.2','Bronze.2']
cnames = list(zip(olympics_data.columns,new_names))

for k,v in cnames:
    col_names[k] = v
    
olympics_data.rename(columns=col_names, inplace=True)
olympics_data

,Country,Summer Olympics,Gold,Silver,Bronze,Winter Olympics,Gold.1,Silver.1,Bronze.1,# Games,Gold.2,Silver.2,Bronze.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Independent Olympic Participants (IOP) [IOP],1,0,1,2,3,0,0,0,0,0,1,0,1,2,3
143,Zambia (ZAM) [ZAM],12,0,1,1,2,0,0,0,0,0,12,0,1,1,2
144,Zimbabwe (ZIM) [ZIM],12,3,4,1,8,1,0,0,0,0,13,3,4,1,8
145,Mixed team (ZZX) [ZZX],3,8,5,4,17,0,0,0,0,0,3,8,5,4,17
